In [1]:
# 繪圖設定字體顏色(否則黑色會看不到)
import matplotlib as mpl
COLOR1 = 'red'
COLOR2 = 'blue'
mpl.rcParams['text.color'] = COLOR1
mpl.rcParams['axes.labelcolor'] = COLOR2
mpl.rcParams['xtick.color'] = COLOR2
mpl.rcParams['ytick.color'] = COLOR2

# 1.OpenCV臉部比對函數

In [17]:
import cv2
import numpy as np
# 灰階讀取影像
images = []
images.append(cv2.imread("data/a1.png", cv2.IMREAD_GRAYSCALE))
images.append(cv2.imread("data/a2.png", cv2.IMREAD_GRAYSCALE))
images.append(cv2.imread("data/b1.png", cv2.IMREAD_GRAYSCALE))
images.append(cv2.imread("data/b2.png", cv2.IMREAD_GRAYSCALE))
# 標示：同一個人標示同一編碼
labels = [0, 0, 1, 1]

# 1.LBP演算法(LBPHFaceRecognizer)
# 訓練
face_recognizer1 = cv2.face.LBPHFaceRecognizer_create()
face_recognizer1.train(images, np.array(labels)) 
# 預測
predict_image = cv2.imread("0.data/Face_Detection with_Deep Learning_in_Keras/a3.png", cv2.IMREAD_GRAYSCALE)
label1, confidence1 = face_recognizer1.predict(predict_image) 
print("LBPHFaceRecognizer label =", label1)
print("LBPHFaceRecognizer confidence =", confidence1)

# 2.EigenFaceRecognizer
face_recognizer2 = cv2.face.EigenFaceRecognizer_create()
face_recognizer2.train(images, np.array(labels)) 
# 預測
label2, confidence2 = face_recognizer2.predict(predict_image) 
print("EigenFaceRecognizer label =", label2)
print("EigenFaceRecognizer confidence =", confidence2)

# 3.FisherFaceRecognizer
face_recognizer3 = cv2.face.FisherFaceRecognizer_create()
face_recognizer3.train(images, np.array(labels)) 
# 預測
label3, confidence3 = face_recognizer3.predict(predict_image) 
print("FisherFaceRecognizer label =", label3)
print("FisherFaceRecognizer confidence =", confidence3)

LBPHFaceRecognizer label = 0
LBPHFaceRecognizer confidence = 67.6856704732354
EigenFaceRecognizer label = 0
EigenFaceRecognizer confidence = 839.5760877931219
FisherFaceRecognizer label = 0
FisherFaceRecognizer confidence = 597.4287226710123


# 2.dlib

In [21]:
# 產生128D numpy特徵向量
import numpy as np
import cv2
import dlib
# 圖片 轉RGB
image = cv2.imread("data/jared_1.jpg")
rgb = image[:, :, ::-1]

# (1)正面人臉偵測器
    # rects = detector(圖像, 0) 回傳值 rectangles[第一張臉[(左上x, 左上y) (右下x, 右下y)], ...]
detector = dlib.get_frontal_face_detector()
# (2)訓練好的 人臉特徵檢測器(5點)：檔案太大，請自行搜尋下載
    # shape = predictor(圖像, rects[i]) 轉numpy 就是 特徵點座標 list
pose_predictor_5_point = dlib.shape_predictor("data/shape_predictor_5_face_landmarks.dat")
# (3)生成面部識別器
    # 特徵點 轉換為 128D面部描述，同一人轉換後特徵位置相近，可用來 臉部比對 是否同一人
    # face_encoder.compute_face_descriptor(圖像, 人臉特徵檢測器shape, num_jitters=如果大於1表示輕度隨機晃動次數)
    # 模型檔下載
        # https://github.com/davisking/dlib-models/blob/master/dlib_face_recognition_resnet_model_v1.dat.bz2
face_encoder = dlib.face_recognition_model_v1("data/dlib_face_recognition_resnet_model_v1.dat")

def face_encodings(face_image, number_of_times_to_upsample=1, num_jitters=1):
    # (1)人臉位置
    face_locations = detector(face_image, number_of_times_to_upsample)
    # (2)人臉特徵(沒有轉numpy) list
    raw_landmarks = [pose_predictor_5_point(face_image, face_location) for face_location in face_locations]
    # (3)128D面部描述 list
    return [np.array(face_encoder.compute_face_descriptor(face_image, raw_landmark_set, num_jitters)) for
            raw_landmark_set in raw_landmarks]
# 所有人臉 128D面部描述 list
    # [第一張臉 numpy shape=(128,), 第二張, ...]
encodings = face_encodings(rgb)
for encoding in encodings:
    print(len(encoding))
    print(type(encoding))

128
<class 'numpy.ndarray'>


In [29]:
# 128D numpy特徵向量 進行人臉比對
import numpy as np
import cv2
import dlib
# 圖片
known_image_1 = cv2.imread("data/jared_1.jpg")
known_image_2 = cv2.imread("data/jared_2.jpg")
known_image_3 = cv2.imread("data/jared_3.jpg")
known_image_4 = cv2.imread("data/obama.jpg")
unknown_image = cv2.imread("data/jared_4.jpg")
# 轉RGB
known_image_1 = known_image_1[:, :, ::-1]
known_image_2 = known_image_2[:, :, ::-1]
known_image_3 = known_image_3[:, :, ::-1]
known_image_4 = known_image_4[:, :, ::-1]
unknown_image = unknown_image[:, :, ::-1]
# 圖片名稱
names = ["jared_1.jpg", "jared_2.jpg", "jared_3.jpg", "obama.jpg"]

# (1)正面人臉偵測器
    # rects = detector(圖像, 0) 回傳值 rectangles[第一張臉[(左上x, 左上y) (右下x, 右下y)], ...]
detector = dlib.get_frontal_face_detector()
# (2)訓練好的 人臉特徵檢測器(5點)
    # shape = predictor(圖像, rects[i]) 轉numpy 就是 特徵點座標 list
pose_predictor_5_point = dlib.shape_predictor("data/shape_predictor_5_face_landmarks.dat")
# (3)生成面部識別器
    # 特徵點 轉換為 128D面部描述，同一人轉換後特徵位置相近，可用來 臉部比對 是否同一人
    # face_encoder.compute_face_descriptor(圖像, 人臉特徵檢測器shape, num_jitters=如果大於1表示輕度隨機晃動次數)
    # 模型檔下載
        # https://github.com/davisking/dlib-models/blob/master/dlib_face_recognition_resnet_model_v1.dat.bz2
face_encoder = dlib.face_recognition_model_v1("data/dlib_face_recognition_resnet_model_v1.dat")

def face_encodings(face_image, number_of_times_to_upsample=1, num_jitters=1):
    # (1)人臉位置
    face_locations = detector(face_image, number_of_times_to_upsample)
    # (2)人臉特徵(沒有轉numpy) list
    raw_landmarks = [pose_predictor_5_point(face_image, face_location) for face_location in face_locations]
    # (3)128D面部描述 list
    return [np.array(face_encoder.compute_face_descriptor(face_image, raw_landmark_set, num_jitters)) for
            raw_landmark_set in raw_landmarks]
# 128D面部描述：只有一張臉 shape=(128,)
known_image_1_encoding = face_encodings(known_image_1)[0]
known_image_2_encoding = face_encodings(known_image_2)[0]
known_image_3_encoding = face_encodings(known_image_3)[0]
known_image_4_encoding = face_encodings(known_image_4)[0]
known_encodings = [known_image_1_encoding, known_image_2_encoding, known_image_3_encoding, known_image_4_encoding]
unknown_encoding = face_encodings(unknown_image)[0]

# 比較距離後排序(小到大)：(距離, 名稱)
    # sorted(iterable, reverse=False改True由大到小排序)
def compare_faces_ordered(encodings, face_names, encoding_to_check):
    distances = list(np.linalg.norm(encodings - encoding_to_check, axis=1))
    return zip(*sorted(zip(distances, face_names)))

# 比較距離：(所有臉的128D面部描述 list, 要比較的那張臉128D面部描述)
def compare_faces(encodings, encoding_to_check):
    # np.linalg.norm 回傳範數(距離)
    return list(np.linalg.norm(encodings - encoding_to_check, axis=1))

computed_distances = compare_faces(known_encodings, unknown_encoding)
computed_distances_ordered, ordered_names = compare_faces_ordered(known_encodings, names, unknown_encoding)

# Print obtained results
print('距離list =', computed_distances)
print('排序後距離list =', computed_distances_ordered)
print('相似圖像排序list =', ordered_names)

距離list = [0.39983264838593896, 0.4104153683230741, 0.3913191431497527, 0.9053700273411349]
排序後距離list = (0.3913191431497527, 0.39983264838593896, 0.4104153683230741, 0.9053700273411349)
相似圖像排序list = ('jared_3.jpg', 'jared_1.jpg', 'jared_2.jpg', 'obama.jpg')


# 3.face_recognition：face_encodings編碼 人臉比對

In [18]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import cv2
import face_recognition
# 載入圖片：RBG
Image1 = face_recognition.load_image_file("data/ciaburro.jpg")
Image2 = face_recognition.load_image_file("data/tiziana.jpg")
UnknownImage = face_recognition.load_image_file("data/tiziana2.jpg")
# face_encodings 分析影像 並計算 128個維度臉部編碼 shape=(128,)
try:
    Image1Encoding = face_recognition.face_encodings(Image1)[0]
    Image2Encoding = face_recognition.face_encodings(Image2)[0]
    UnknownImageEncoding = face_recognition.face_encodings(UnknownImage)[0]
except IndexError:
    print("Any face was located. Check the image files..")
    quit()
# 已知臉部list 轉編碼，比較第三張臉
    # results 回傳 [是否 1st的臉, ...]
known_faces = [Image1Encoding, Image2Encoding]
results = face_recognition.compare_faces(known_faces, UnknownImageEncoding)

print("Is the unknown face a picture of ciaburro? {}".format(results[0]))
print("Is the unknown face a picture of Tiziana? {}".format(results[1]))
print("Is the unknown face a new person that we've never seen before? {}".format(not True in results))

Is the unknown face a picture of ciaburro? False
Is the unknown face a picture of Tiziana? True
Is the unknown face a new person that we've never seen before? False
